In [1]:
# imports
import pandas as pd
from datetime import datetime
import json 
import numpy as np 
import os 
import requests

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
cb_df = pd.read_csv('/Users/Oswal/Documents/GitHub/Project-Week-3/notebooks/Toulouse_Data.csv')

In [3]:
cb_df.head()

,Station Name,Address,empty_slots,free_bikes,latitude,longitude,timestamp,Unique ID
0,00197 - CASERNE PERIGNON,6 AV LOUIS BLERIOT,19,1,43.600386,1.464094,2022-12-05T00:29:57.070000Z,197
1,00213 - PUJOL - CHAUBET,129 AV DE CASTRES,16,1,43.600755,1.467514,2022-12-05T00:29:57.109000Z,213
2,00055 - SAINT-SERNIN - G. ARNOULT,2 RUE GATIEN ARNOULT,4,11,43.608952,1.441004,2022-12-05T00:29:56.992000Z,55
3,00195 - LARROUSSE - RIEUX,FACE 15 RUE PIERRE LAROUSSE,13,5,43.597235,1.459071,2022-12-05T00:29:56.994000Z,195
4,00280 - CHARBONNIERE,RUE DE LA CHARBONNIERE ENTRE IMP AMPERE ET RUE...,8,10,43.573830,1.448710,2022-12-05T00:29:56.996000Z,280


In [4]:
cb_df.info

<bound method DataFrame.info of                           Station Name  \
0             00197 - CASERNE PERIGNON   
1              00213 - PUJOL - CHAUBET   
2    00055 - SAINT-SERNIN - G. ARNOULT   
3            00195 - LARROUSSE - RIEUX   
4                 00280 - CHARBONNIERE   
..                                 ...   
281       00239 - LESPINET - HERBETTES   
282    00193 - LOMBEZ - PETITE VITESSE   
283          00007 - PLACE VICTOR HUGO   
284                 00040 - COLOMBETTE   
285                     00029 - VALADE   

                                               Address  empty_slots  \
0                                   6 AV LOUIS BLERIOT           19   
1                                    129 AV DE CASTRES           16   
2                                 2 RUE GATIEN ARNOULT            4   
3                          FACE 15 RUE PIERRE LAROUSSE           13   
4    RUE DE LA CHARBONNIERE ENTRE IMP AMPERE ET RUE...            8   
..                                   

In [5]:
cb_df.shape

(286, 8)

In [6]:
station_name = cb_df['Station Name']
address = cb_df['Address']
empty_slots = cb_df['empty_slots']
free_bikes = cb_df['free_bikes']
longitude = cb_df['longitude']
latitude = cb_df['latitude']
timestamp = cb_df['timestamp']
id = cb_df['Unique ID']

In [7]:
print(latitude[0],longitude[0])
print(station_name[0])

43.60038605818475 1.464093716152017
00197 - CASERNE PERIGNON


In [8]:
api_key= 'API_KEY'

In [9]:
url = 'https://api.foursquare.com/v3/places/search?ll=' + str(latitude[0]) + ',' + str(longitude[0]) +'&radius=1000'+'&categories=13003%2C%2013032%2C%2018062%2C%2010040%20'
print(url)
headers = {"Accept":"application/json"}
headers['Authorization'] = api_key
result = requests.get(url,headers = headers)

foursquare_data = result.json()
foursquare_data['results'][0]['categories'][0]['name']

https://api.foursquare.com/v3/places/search?ll=43.60038605818475,1.464093716152017&radius=1000&categories=13003%2C%2013032%2C%2018062%2C%2010040%20


'Beer Bar'

In [10]:
fsquare_list = []
for i in range(len(latitude)):
    url = 'https://api.foursquare.com/v3/places/search?ll=' + str(latitude[i]) + ',' + str(longitude[i]) +'&radius=1000'+'&categories=13003%2C%2013032%2C%2018062%2C%2010040%20'
    headers = {"Accept":"application/json"}
    headers['Authorization'] = api_key
    result_test = requests.get(url, headers=headers)
    fsquare_list.append(result_test.json())

In [17]:
def hotspots_filter(df):
    hotspots = []
    for i in range(len(df['results'])):
                hotspots.append(df['results'][i])
                break
    return hotspots

In [18]:
hotspots_data =[]
for i in range(len(fsquare_list)):
    hotspots_data.append(hotspots_filter(fsquare_list[i]))


In [47]:

venue_data = pd.DataFrame(columns=['name','country','categories','near_station'])
venue_data

,name,country,categories,near_station


Put your parsed results into a DataFrame

In [73]:
def data_extract(df, venue_data, cb_df):
    for i in range(len(venue_data)):
        for n in range(len(venue_data[i])):
            name = venue_data[i][n]['name']
            country = venue_data[i][n]['location']['country']
            category = venue_data[i][n]['categories']
            near_station = cb_df['Station Name'][i]
            info = pd.Series({'name':name, 'country':country, 'categories':category, 'near_station':near_station})
            print(info)
            df = pd.concat([df,info.to_frame().T],ignore_index=True)
    
    return df

In [74]:
venue_data = data_extract(venue_data, hotspots_data, cb_df)

name                                                      De Danu
country                                                        FR
categories      [{'id': 13006, 'name': 'Beer Bar', 'icon': {'p...
near_station                             00197 - CASERNE PERIGNON
dtype: object
name                                                      De Danu
country                                                        FR
categories      [{'id': 13006, 'name': 'Beer Bar', 'icon': {'p...
near_station                              00213 - PUJOL - CHAUBET
dtype: object
name                                      Les Délices de Saturnin
country                                                        FR
categories      [{'id': 13035, 'name': 'Coffee Shop', 'icon': ...
near_station                    00055 - SAINT-SERNIN - G. ARNOULT
dtype: object
name                                                      De Danu
country                                                        FR
categories      [{'id': 13006, 'na

In [75]:
venue_data.to_csv('foursquare_venue.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [81]:
print(latitude[0],longitude[0])
print(station_name[0])

43.60038605818475 1.464093716152017
00197 - CASERNE PERIGNON


In [77]:
yelp_api_key = 'API_KEY'

In [86]:
yelp_url = 'https://api.yelp.com/v3/businesses/search?' + 'latitude=' + str(latitude[0]) + '&longitude=' + str(longitude[0]) + '&radius=1000&categories=football%2Cbars%2Cjazzandblues%2Ccafes&sort_by=best_match&limit=20'
headers = {"accept": "application/json"}
headers['Authorization'] = 'Bearer ' + yelp_api_key
response_yelp = requests.get(yelp_url, headers = headers)
print(response_yelp.text)


{"businesses": [{"id": "SU3dxcSBVxG4AiXO0V8QMw", "alias": "de-dan\u00fa-toulouse", "name": "De Dan\u00fa", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/crV-aVB_d8FEPpekzahRGA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/de-dan%C3%BA-toulouse?adjust_creative=Jz48_hIhAC1KqmvqmPZMpQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Jz48_hIhAC1KqmvqmPZMpQ", "review_count": 84, "categories": [{"alias": "irish_pubs", "title": "Irish Pub"}], "rating": 4.0, "coordinates": {"latitude": 43.6007, "longitude": 1.4558899}, "transactions": [], "price": "\u20ac\u20ac", "location": {"address1": "9 rue du Pont Guilhemery", "address2": null, "address3": null, "city": "Toulouse", "zip_code": "31000", "country": "FR", "state": "31", "display_address": ["9 rue du Pont Guilhemery", "31000 Toulouse", "France"]}, "phone": "+33561625879", "display_phone": "+33 5 61 62 58 79", "distance": 663.219433565249}, {"id": "UihrYqB3iGkKaESMfpvbmA", "alias": "la-dynamo-toulous

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [88]:
yelp_list = []
for i in range(len(latitude)):
    yelp_url = 'https://api.yelp.com/v3/businesses/search?' + 'latitude=' + str(latitude[i]) + '&longitude=' + str(longitude[i]) + '&radius=1000&categories=football%2Cbars%2Cjazzandblues%2Ccafes&sort_by=best_match&limit=20'
    headers = {"Accept":"application/json"}
    headers['Authorization'] = 'Bearer ' + yelp_api_key
    response_yelp = requests.get(yelp_url, headers=headers)
    yelp_list.append(response_yelp.json())


SSLError: HTTPSConnectionPool(host='api.yelp.com', port=443): Max retries exceeded with url: /v3/businesses/search?latitude=43.5714330640513&longitude=1.46278065695466&radius=1000&categories=football%2Cbars%2Cjazzandblues%2Ccafes&sort_by=best_match&limit=20 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1131)')))

In [89]:
yelp_list[0]['businesses'][0]

{'id': 'SU3dxcSBVxG4AiXO0V8QMw',
 'alias': 'de-danú-toulouse',
 'name': 'De Danú',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/crV-aVB_d8FEPpekzahRGA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/de-dan%C3%BA-toulouse?adjust_creative=Jz48_hIhAC1KqmvqmPZMpQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Jz48_hIhAC1KqmvqmPZMpQ',
 'review_count': 84,
 'categories': [{'alias': 'irish_pubs', 'title': 'Irish Pub'}],
 'rating': 4.0,
 'coordinates': {'latitude': 43.6007, 'longitude': 1.4558899},
 'transactions': [],
 'price': '€€',
 'location': {'address1': '9 rue du Pont Guilhemery',
  'address2': None,
  'address3': None,
  'city': 'Toulouse',
  'zip_code': '31000',
  'country': 'FR',
  'state': '31',
  'display_address': ['9 rue du Pont Guilhemery', '31000 Toulouse', 'France']},
 'phone': '+33561625879',
 'display_phone': '+33 5 61 62 58 79',
 'distance': 663.219433565249}

Put your parsed results into a DataFrame

In [107]:
yelp_venue_data= pd.DataFrame(columns=['name','country','city','categories','review_count','rating','near_station'])
yelp_venue_data

,name,country,city,categories,review_count,rating,near_station


In [112]:
def data_extract_yelp(df, yelp_venue_data, cb_df):
    for i in range(len(yelp_venue_data)):
        for n in range(len(yelp_venue_data[i]['businesses'])):
            name = yelp_venue_data[i]['businesses'][n]['name']
            country = yelp_venue_data[i]['businesses'][n]['location']['country']
            city = yelp_venue_data[i]['businesses'][n]['location']['city']
            categories = yelp_venue_data[i]['businesses'][n]['categories']
            review_count = yelp_venue_data[i]['businesses'][n]['review_count']
            rating = yelp_venue_data[i]['businesses'][n]['rating']
            near_station = cb_df['Station Name'][i]
            info = pd.Series({'name':name, 
                              'country':country, 
                              'city':city,
                              'categories': categories, 
                              'review_count':review_count,
                              'rating':rating,
                              'near_station':near_station})
            df = pd.concat([df,info.to_frame().T],ignore_index=True)
    return df


In [113]:
yelp_venue_data = data_extract_yelp(yelp_venue_data, yelp_list, cb_df)

In [114]:
yelp_venue_data

,name,country,city,categories,review_count,rating,near_station
0,De Danú,FR,Toulouse,"[{'alias': 'irish_pubs', 'title': 'Irish Pub'}]",84,4.0,00197 - CASERNE PERIGNON
1,La Dynamo,FR,Toulouse,"[{'alias': 'danceclubs', 'title': 'Dance Clubs...",31,4.5,00197 - CASERNE PERIGNON
2,Au Poêle de la Bête,FR,Toulouse,"[{'alias': 'tapas', 'title': 'Tapas Bars'}, {'...",18,4.5,00197 - CASERNE PERIGNON
3,L'Heure du Singe,FR,Toulouse,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",17,4.5,00197 - CASERNE PERIGNON
4,Puerto Habana,FR,Toulouse,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",33,3.5,00197 - CASERNE PERIGNON
...,...,...,...,...,...,...,...
95,Le Seize,FR,Toulouse,"[{'alias': 'brasseries', 'title': 'Brasseries'...",3,3.0,00242 - RAIMBAUD - COLETTE
96,Le Seizième,FR,Toulouse,"[{'alias': 'bars', 'title': 'Bars'}]",1,4.0,00242 - RAIMBAUD - COLETTE
97,Karting Café,FR,Toulouse,"[{'alias': 'cafes', 'title': 'Cafes'}]",2,3.0,00242 - RAIMBAUD - COLETTE
98,Les Oliviers,FR,Toulouse,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",1,5.0,00224 - CAMPUS SUPAERO


In [115]:
yelp_venue_data.to_csv('yelp_venue.csv', index = False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Both API's provided reliable data but differed in the insights they offer. Foursquare offered a greater volume of rows or raw data but did not offer much details in the way of parameters. Yelp API offered inverse results where it return less quantity of overall data overall, but for the data it did return, it offered greater context/detail through columns like review count and rating.

Get the top 10 restaurants according to their rating

In [119]:
yelp_venue_data.sort_values(by=['rating'], ascending=[False]).head(10)

,name,country,city,categories,review_count,rating,near_station
98,Les Oliviers,FR,Toulouse,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",1,5.0,00224 - CAMPUS SUPAERO
37,Animaé Bar,FR,Toulouse,"[{'alias': 'bars', 'title': 'Bars'}]",1,5.0,00213 - PUJOL - CHAUBET
36,Chez Nous,FR,Toulouse,"[{'alias': 'bars', 'title': 'Bars'}]",1,5.0,00213 - PUJOL - CHAUBET
30,Le Tartacs,FR,Toulouse,"[{'alias': 'bars', 'title': 'Bars'}]",2,5.0,00213 - PUJOL - CHAUBET
33,La Salsathèque,FR,Toulouse,"[{'alias': 'dance_schools', 'title': 'Dance Sc...",1,5.0,00213 - PUJOL - CHAUBET
70,Les Complices,FR,Toulouse,"[{'alias': 'french', 'title': 'French'}, {'ali...",11,4.5,00195 - LARROUSSE - RIEUX
57,Bar L'Autan,FR,Toulouse,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",16,4.5,00055 - SAINT-SERNIN - G. ARNOULT
21,La Dynamo,FR,Toulouse,"[{'alias': 'danceclubs', 'title': 'Dance Clubs...",31,4.5,00213 - PUJOL - CHAUBET
67,L'Heure du Singe,FR,Toulouse,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",17,4.5,00195 - LARROUSSE - RIEUX
66,La Dynamo,FR,Toulouse,"[{'alias': 'danceclubs', 'title': 'Dance Clubs...",31,4.5,00195 - LARROUSSE - RIEUX
